<a href="https://colab.research.google.com/github/The-Bambi/LSTM-text-generator/blob/master/Krystian_FIgiel_projekt_kodeks_karny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Krystian Figiel - generowanie tekstu z kodeksu karnego

In [0]:
import pandas as pd
import numpy as np

import keras
from keras import models
from keras.layers import *
from keras.utils import to_categorical
from tensorflow import nn

import matplotlib.pyplot as plt
from random import randint

import re

Using TensorFlow backend.


# Przygotowywanie zestawu danych
- usuwanie niepotrzebnych znaków, 
- usuwanie dat
- zastępowanie polskich znaków

w celu zmniejszenia wektora znaków do generowania - wielkość tego wektora bardzo wpływa na czas i skuteczność uczenia

In [0]:
file = open('kodeks.txt','r')

In [0]:
def clear(line):
  i = line.find('©')
  if i >= 0:
    return ""
  j = line.find('s. ')
  if j >= 0:
    return ""
  cleared = line.replace('\n',' ').replace('§','').lower().replace('ł','l').replace('ą','a').replace('ę','e').replace('ż','z').replace('ź','z').replace('ń','n').replace('ó','o').replace('ś','s').replace('ć','c').replace("\x0c","").replace("”",chr(34)).replace("„",chr(34)).replace("–","-")
  return cleared

In [0]:
everything = ''

for line in file:
  everything += clear(line)
file.close()

Regular Expressions jako łatwy sposób na usuwanie konkretnych ciągów znaków - w tym przypadku dat i powtarzających się spacji.

In [0]:
everything = re.sub(' +', ' ', everything)
everything = re.sub('^(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4}|am|pm)$', '', everything)

In [0]:
signs_set = set(everything)
signs_to_ints = {}
ints_to_signs = {}
for i,s in enumerate(sorted(signs_set)):
  ints_to_signs[i], signs_to_ints[s] = s, i

Słowniki *ints_to_signs* i *signs_to_ints* służą do wektoryzacji liter i do odkodowania wektorów liter. Każdemu znakowi przypisuje się numer, który potem służy do stworzenia tablicy numpy wypełnionej zerami i 1 na odpowiednim indeksie.

In [0]:
len(everything), len(signs_set)

(248985, 46)

Ostateczna długość całego tekstu to 248985 znaków ze zbioru 46 unikatowych.

Następny segment to podział znaków na zbiór uczący i etykiety. LSTM na wejściu bierze wektor 3D. W celu utworzenia takiego zbioru należy podzielić cały tekst na odcinki po N liter, i każdemu takiemu odcinkowi przypisać etykietę w postaci kolejnej litery, która występuję w tekście po tym ciągu znaków. Sieć nauczy się generować literę na podstawie ciągu znaków.

In [0]:
maxlen = 40
step = 1
sentences = []
next_chars = []
for i in range(120000, len(everything) - maxlen, step):
    sentences.append(everything[i: i + maxlen])
    next_chars.append(everything[i + maxlen])

#Wektoryzacja liter:

In [0]:
x = np.zeros((len(sentences), maxlen, len(signs_set)))
y = np.zeros((len(sentences), len(signs_set)))

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, signs_to_ints[char]] = 1
    y[i, signs_to_ints[next_chars[i]]] = 1

Zbiór X to trójwymiarowa tablica numpy. W tym przypadku ma wymiary (128935, 40, 46), czyli 128935 "zdań" po 40 znaków, gdzie każdy znak jest przedstawiony wektorowo.

# Budowa modelu.
Użyłem warstwy CuDNNLSTM, czyli implementacji LSTM w technologii CUDA, która została napisana specjalnie do obliczeń na kartach graficznych. Przyspiesza to uczenie sieci ~10-krotnie. Na wyjściu musi się znaleźć warstwa w pełni połączona o wielkości odpowiadającej długości zbioru unikatowych znaków. Metodą prób i błędów odkryłem, że sieć uczy się bardzo skutecznie po połączeniu warstwy LSTM i wyjściowej jeszcze jedną warstwą w pełni połączoną.

In [0]:
model = models.Sequential()
model.add(CuDNNLSTM(128, input_shape=(maxlen, len(signs_set)), return_sequences=False))
model.add(Dense(128))
model.add(Dense(len(signs_set), activation=nn.softmax))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


# Uczenie modelu
W tym przypadku model potrzebuje stosunkowo dużej ilości epok, a zjawisko przeuczenia (widoczne jako rosnąca strata zbioru walidacyjnego) nie jest niemile widziane - w pewnym sensie model powinien się nauczyć "na pamięć". Po więcej niż ~50 epokach strata zaczyna rosnąć.

In [0]:
a = 0
b = 120
model.fit(x[a*1000:b*1000], y[a*1000:b*1000], batch_size=128, epochs=50, validation_split=0.2)

Generowanie tekstu rozpoczyna się od wrzucenie do sieci "seed", czyli początkowego fragmentu tekstu, który będzie podstawą dla sieci do wygenerowania kolejnej litery. Następnie wygenerowana litera jest dodana do końca zdania, z którego również ucina się pierwszą literę. Taką czynność powtarza się tyle razy ile chcemy, żeby sieć wygenerowała tekstu. Od pewnego momentu sieć działa wyłącznie na wygenerowanym przez siebie tekście.

In [0]:
start_index = randint(0, len(everything) - maxlen - 1)
sentence = everything[start_index: start_index + maxlen]

generated = ''
generated += "[{}]".format(sentence)
file = open("generated.txt","a")

for i in range(500):
    x_pred = np.zeros((1, maxlen, len(signs_set)))
    for t, char in enumerate(sentence):
        x_pred[0, t, signs_to_ints[char]] = 1.

    preds = model.predict(x_pred, verbose=0)
    next_index = np.argmax(preds)
    next_char = ints_to_signs[next_index]

    sentence = sentence[1:] + next_char
    generated += next_char
    
generated += '\n'
print(generated)
file.write(generated)
file.close()

[k na zdrowiu czlowieka, sprawca podlega ]karze pozbawienia wolnosci od 3 miesiecy do lat 5. art. 277bia 16 lu art. 271a 1 lub 2 albo art. 277c art. 233 skazania czynnosci wymagajacych z handlem osobe na podstawie wyroku trybunalu konstytucyjnego zamach albo istotnie lub informacji lub taka jako taki albo doprowadzajac czlowieka do stanu nieprawnien orzeczenia sadu lub innym powinienie lub uszkadzajac czynu zabronionego okreslonego w art. 233 i 2000 srost albo o glosowanie w okreslony sposob moze to dzialajacej w wysokosci do samodziele



#Generowanie z temperaturą

Wygenerowaną tablicę można zmienić, dzięki czemu uzyskuje się bardziej zróżnicowane wyniki, co prowadzi też do bardziej zróżnicowanych przewidywań.


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
for _ in range(20):
  start_index = randint(0, len(everything) - maxlen - 1)

  file = open("generated.txt","a")

  for diversity in [0.2, 0.5, 1.0, 1.2]:
      generated = ''
      sentence = everything[start_index: start_index + maxlen]
      generated += "[{}]".format(sentence)

      for i in range(500):
          x_pred = np.zeros((1, maxlen, len(signs_set)))
          for t, char in enumerate(sentence):
              x_pred[0, t, signs_to_ints[char]] = 1.

          preds = model.predict(x_pred, verbose=0)[0]
          next_index = sample(preds, diversity)
          next_char = ints_to_signs[next_index]

          sentence = sentence[1:] + next_char
          generated += next_char

  generated += '\n'
  print(generated)
  file.write(generated)
  file.close()

#Bibliografia
Keras examples directory - https://github.com/keras-team/keras/tree/master/examples [10.06.2019]